In [ ]:
import gpxpy
from shapely.geometry import Point

with open("first_run.gpx", "r") as gpx_contents:
    gpx = gpxpy.parse(gpx_contents)

run_points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            run_points.append(Point(point.longitude, point.latitude))

In [ ]:
from utils import display_points

display_points(run_points)

In [ ]:
import numpy as np
from shapely.geometry import Point, LineString

num_points = 1000
run_line = LineString(run_points)
distances = np.linspace(0, run_line.length, num=num_points)
run_points = [
    Point(run_line.interpolate(distance).coords[0])
    for distance in distances
]

In [ ]:
from utils import display_points

display_points(run_points)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

DATABASE_URL = "postgresql://vinayak@localhost:5432/paris"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Float, Integer, String
from geoalchemy2 import Geometry

class Street(Base):
    __tablename__ = "streets"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    geometry = Column(Geometry("MULTILINESTRING", srid=4326))

Base.metadata.create_all(engine)

In [ ]:
streets = session.query(Street).all()
print(f"Found {len(streets)} streets in database")

In [ ]:
from sqlalchemy import func
from geoalchemy2 import WKTElement
from geoalchemy2.shape import to_shape

def match_points_to_streets(run_points):
    street_points = {}
    for point in run_points:
        nearest_street = (
            session.query(Street)
            .order_by(
                func.ST_Distance(
                    Street.geometry,
                    WKTElement(point.wkt, srid=4326)
                )
            ).first()
        )
    
        if nearest_street:
            if nearest_street.name not in street_points:
                street_points[nearest_street.name] = { "street": nearest_street, "points": [] }
    
            street_geom = to_shape(nearest_street.geometry)
            projected_point = street_geom.interpolate(
                street_geom.project(point)
            )

            street_points[nearest_street.name]["points"].append(
                projected_point
            )

    return street_points

street_points = match_points_to_streets(run_points)

In [ ]:
street_names = street_points.keys()
street_names

In [ ]:
from utils import display_points

all_matched_points = []
for k, v in street_points.items():
    all_matched_points.extend(v['points'])

display_points(all_matched_points)

In [ ]:
class CoveredStreet(Base):
    __tablename__ = "covered_streets"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    geometry = Column(Geometry("MULTILINESTRING", srid=4326))
    progress_percentage = Column(Float)

Base.metadata.create_all(engine)

In [ ]:
from utils import convert_points_to_linestring, clip_linestring_to_street

for street_name, data in street_points.items():
    street_linestring = to_shape(data['street'].geometry)
    points = data['points']

    covered_linestring = convert_points_to_linestring(points)
    covered_linestring = clip_linestring_to_street(covered_linestring, street_linestring)

    progress = (
        covered_linestring.length / street_linestring.length
    ) * 100

    covered_street = CoveredStreet(
        name=street_name,
        geometry=WKTElement(covered_linestring.wkt, srid=4326),
        progress_percentage=progress
    )
    session.add(covered_street)

    print(f"{street_name}: {progress}")

session.commit()

In [ ]:
from utils import display_covered_streets

covered_streets = session.query(CoveredStreet).all()
display_covered_streets(covered_streets)

In [ ]:
total_streets = session.query(Street).count()
covered_streets = session.query(CoveredStreet).filter(
    CoveredStreet.progress_percentage >= 90
).count()

print(f"Progress: {covered_streets} / {total_streets} streets")

In [ ]:
import gpxpy
from shapely.geometry import Point

with open("second_run.gpx", "r") as gpx_contents:
    gpx = gpxpy.parse(gpx_contents)

run_points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            run_points.append(Point(point.longitude, point.latitude))

In [ ]:
from utils import display_points

display_points(run_points)

In [ ]:
import numpy as np
from shapely.geometry import Point, LineString

num_points = 1000
run_line = LineString(run_points)
distances = np.linspace(0, run_line.length, num=num_points)
run_points = [
    Point(run_line.interpolate(distance).coords[0])
    for distance in distances
]

In [ ]:
from utils import display_points

display_points(run_points)

In [ ]:
from sqlalchemy import func
from geoalchemy2 import WKTElement
from geoalchemy2.shape import to_shape

def match_points_to_streets(run_points):
    street_points = {}
    for point in run_points:
        nearest_street = (
            session.query(Street)
            .order_by(
                func.ST_Distance(
                    Street.geometry,
                    WKTElement(point.wkt, srid=4326)
                )
            ).first()
        )
    
        if nearest_street:
            if nearest_street.name not in street_points:
                street_points[nearest_street.name] = { "street": nearest_street, "points": [] }
    
            street_geom = to_shape(nearest_street.geometry)
            projected_point = street_geom.interpolate(
                street_geom.project(point)
            )
    
            street_points[nearest_street.name]["points"].append(
                projected_point
            )

    return street_points

street_points = match_points_to_streets(run_points)

In [ ]:
street_names = street_points.keys()
street_names

In [ ]:
from utils import display_points

all_matched_points = []
for k, v in street_points.items():
    all_matched_points.extend(v['points'])

display_points(all_matched_points)

In [ ]:
from utils import combine_linestrings, convert_points_to_linestring

for street_name, data in street_points.items():
    street_linestring = to_shape(data['street'].geometry)
    points = data['points']

    existing_covered_street = (
        session.query(CoveredStreet).filter(
            CoveredStreet.name == street_name
        ).first()
    )

    covered_linestring = convert_points_to_linestring(points)
    covered_linestring = clip_linestring_to_street(covered_linestring, street_linestring)

    if existing_covered_street:
        existing_linestring = to_shape(
            existing_covered_street.geometry
        )

        combined_linestring = combine_linestrings(
            existing_linestring,
            covered_linestring
        )
        progress = (
            combined_linestring.length / street_linestring.length
        ) * 100
        existing_covered_street.geometry = WKTElement(
            combined_linestring.wkt, srid=4326
        )
        existing_covered_street.progress_percentage = progress

        print(f"{street_name}: {progress}")

session.commit()

In [ ]:
from utils import display_covered_streets

covered_streets = session.query(CoveredStreet).all()
display_covered_streets(covered_streets)

In [ ]:
total_streets = session.query(Street).count()
covered_streets = session.query(CoveredStreet).filter(
    CoveredStreet.progress_percentage >= 90
).count()

print(f"Progress: {covered_streets} / {total_streets} streets")

In [ ]:
covered_streets = session.query(CoveredStreet).all()
for covered_street in covered_streets:
    print(f"{covered_street.name}: {covered_street.progress_percentage}")